# Images Processing
Here three images datasets are processed in order to select the images to be used as input during experiment. A total of three datasets containing screenshots are analyzed. The screenshots were taken after reconstructing the participant's walk from a free exploration walk in a virtual city. Both head and eye tracking data were used and the images where taken using the eye tracking data. Images were saved to a "screenshots" folder with a respective csv file containing the information for each image that was taken.

**TODO:**
- _Is there  anything interesting in the EYE-Local data?_

## 2. Preprocessing
When looking into the `df_b81` file, we found its dimensions are 163661 x 24, and it contains missing values which should be fixed or deleted.

In [1]:
import numpy as np
import os
import shutil

In [ ]:
path = ''
path_face = path + '/faces'
path_objects = path + '/objects'

if not os.path.isdir(path_face):
    os.mkdir(path_face)

if not os.path.isdir(path_objects):
    os.mkdir(path_objects)

for file in os.listdir(path):
    if file.lower().endswith('.png'):
        fileparts = file.split('.')
        if 'face' in fileparts[5]:
            shutil.move(path + '/' + file, path_face + '/' + file)
            print(f'Moving file {file} to folder {path_face}')
        else:
            shutil.move(path + '/' + file, path_objects + '/' + file)
            print(f'Moving file {file} to folder {path_objects}')